In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [13]:
## Alkynes
# M = markovnikov, N = anti-markovnikov
# Requires [Si] marker for M/N or carbene reactions

# TODO: acetylide reactions

RED = AllChem.ReactionFromSmarts('[C:1]#[C:2].[H][H]>>[C:1][C:2]')

# stereochemistry not controlled by ['\', '/']?
RED_LIND = AllChem.ReactionFromSmarts('[C,H:1][C:2]#[C:3][C,H:4].[H][H]>>[R:1]\[C:2]=[C:3]/[R:4]')
RED_BIRCH = AllChem.ReactionFromSmarts('[C,H:1][C:2]#[C:3][C,H:4]>>[R:1]\[C:2]=[C:3]\[R:4]')

X2 = AllChem.ReactionFromSmarts('[C:1]#[C:2].[Cl,Br:3][Cl,Br:4]>>[C:1]([X:3])([X:3])[C:2]([X:4])([X:4])')

HX = AllChem.ReactionFromSmarts('[Si:1]#[C:2].[Cl,Br,I:3]>>[C:1]([X:3])([X:3])[C:2]')

H2O = AllChem.ReactionFromSmarts('[Si:1]#[C:2][H:3].[O:4]>>[C:1](=[O:4])[C:2][H:3]')


alkyne_MN_reactions_list = [
    ('HX', HX, ['Cl', 'Br', 'I'], ['']),
    ('H2O', H2O, ['O'], ['[Hg+2].[O-]S([O-])(=O)=O.OS(O)(=O)=O']),
    
]

alkyne_reactions_list = [
    
    ('RED', RED, ['[H][H]'], ['[Pt]', '[Pd]', '[Ni]', '']),
    ('RED_LIND', RED_LIND, ['[H][H]'], ['[Pd].[Ba+2].[O-]S([O-])(=O)=O']),
    ('RED_BIRCH', RED_BIRCH, [''], ['[Na].N']),
    
    ('X2', X2, ['ClCl', 'BrBr'], ['', 'ClCCl']),
]

In [15]:
import random, copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else cano(smiles)

# choose alkyne
alkyne_smi = 'CC(C)C#CCC'
alkyne = Chem.MolFromSmiles(alkyne_smi)
alkyne_MN = copy.deepcopy(alkyne)

# mark [Si]
C_1, C_2 = alkyne_MN.GetSubstructMatches(Chem.MolFromSmiles('C#C'))[0]
alkyne_MN.GetAtomWithIdx((C_2 if (alkyne_MN.GetAtomWithIdx(C_1).GetTotalNumHs() \
        > alkyne_MN.GetAtomWithIdx(C_2).GetTotalNumHs()) else C_1)).SetAtomicNum(14);

# MN reactions
for reaction in alkyne_MN_reactions_list:
    print reaction[0]
    for reagent in reaction[2]:
        if reagent == '': products = reaction[1].RunReactants((alkyne_MN,))
        else: products = reaction[1].RunReactants((alkyne_MN, Chem.MolFromSmiles(reagent)))
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print cano(alkyne_smi) + block('.', reagent) + '>' + sub_reagent + '>' + '.'.join(product_smi)

# general reactions
for reaction in alkyne_reactions_list:
    print reaction[0]
    for reagent in reaction[2]:
        if reagent == '': products = reaction[1].RunReactants((alkyne,))
        else: products = reaction[1].RunReactants((alkyne, Chem.MolFromSmiles(reagent)))
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        for sub_reagent in reaction[3]:
            print cano(alkyne_smi) + block('.', reagent) + '>' + sub_reagent + '>' + '.'.join(product_smi)

HX
CCC#CC(C)C.Cl>>CCCC(Cl)(Cl)C(C)C
CCC#CC(C)C.Br>>CCCC(Br)(Br)C(C)C
CCC#CC(C)C.I>>CCCC(I)(I)C(C)C
RED
CCC#CC(C)C.[H][H]>[Pt]>CCCCC(C)C
CCC#CC(C)C.[H][H]>[Pd]>CCCCC(C)C
CCC#CC(C)C.[H][H]>[Ni]>CCCCC(C)C
CCC#CC(C)C.[H][H]>>CCCCC(C)C
RED_LIND
CCC#CC(C)C.[H][H]>[Pd].[Ba+2].[O-]S([O-])(=O)=O>CCC=CC(C)C
RED_BIRCH
CCC#CC(C)C>[Na].N>CCC=CC(C)C
X2
CCC#CC(C)C.ClCl>>CCC(Cl)(Cl)C(Cl)(Cl)C(C)C
CCC#CC(C)C.ClCl>ClCCl>CCC(Cl)(Cl)C(Cl)(Cl)C(C)C
CCC#CC(C)C.BrBr>>CCC(Br)(Br)C(Br)(Br)C(C)C
CCC#CC(C)C.BrBr>ClCCl>CCC(Br)(Br)C(Br)(Br)C(C)C
